In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
#pip install xgboost

In [20]:
# Read csv files. And keep a copy of the original dataframes. tran_df and test_df will be used throughout this notebook for preprocessing and scaling.we need original test dataframe in the end
or_train_df = pd.read_csv('train.csv')
or_test_df = pd.read_csv('test.csv')

train_df = or_train_df
test_df = or_test_df

In [21]:
# Dropping 'Id' column as it has no use in training
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [22]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 80)
Shape of test set: (1459, 79)


In [23]:
str_columns = []
num_columns = []

def convert_to_str(df):
    for col in df.columns:
        if (df[col].dtype == np.int64 or df[col].dtype == np.float64):
            df[col] = df[col].fillna(method='ffill')
            if col not in num_columns:
                num_columns.append(col)
        else:
            df[col] = df[col].fillna(method='ffill')
            if col not in str_columns:
                str_columns.append(col)
    return df

train_df = convert_to_str(train_df)
test_df = convert_to_str(test_df)

/tmp/ipython-input-23-656724134.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')
/tmp/ipython-input-23-656724134.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')


In [24]:
train_df['type'] = 'train'
test_df['type'] = 'test'

# Add a dummy SalePrice column to test dataframe to make number of columns equal
test_df['SalePrice'] = train_df['SalePrice'].iloc[:201]

df = train_df._append(test_df, ignore_index=True)

In [25]:
for col in str_columns:
    one_hot = pd.get_dummies(df[col])

    replace_cols = {}
    for one_col in one_hot.columns:
        replace_cols[one_col] = f"{col}_{one_col}"
    one_hot = one_hot.rename(columns=replace_cols)

    df = df.drop(col, axis = 1)
    df = df.join(one_hot)

In [26]:
train_df = df[df['type'] == 'train']
test_df = df[df['type'] == 'test']

train_df = train_df.drop(['type'], axis=1)
test_df = test_df.drop(['type'], axis=1)

test_df = test_df.reset_index(drop=True)

In [27]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 288)
Shape of test set: (1459, 288)


In [28]:
scaler = StandardScaler()
scaler.fit(train_df[num_columns])

StandardScaler()

In [29]:
train_df[num_columns] = scaler.transform(train_df[num_columns])
test_df[num_columns] = scaler.transform(test_df[num_columns])

In [30]:
print(f"Shape of train set: {train_df.shape}")
print(f"Shape of test set: {test_df.shape}")

Shape of train set: (1460, 288)
Shape of test set: (1459, 288)


In [31]:
test_df = test_df.drop(['SalePrice'], axis=1)

In [32]:
train_labels = train_df['SalePrice']
train_data = train_df.drop(['SalePrice'], axis=1)

In [33]:
#XGBOOST REGRESSOR MODEL

In [34]:
model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [35]:
model.fit(train_data, train_labels)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.1, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [36]:
predictions = model.predict(test_df)

In [37]:
test_df['SalePrice'] = predictions
test_df[num_columns] = scaler.inverse_transform(test_df[num_columns])
test_df = pd.DataFrame(test_df, columns=train_df.columns)

In [38]:
# Create results dataframe
results = pd.DataFrame()
results['Id'] = or_test_df['Id']
results['SalePrice'] = test_df['SalePrice']

In [39]:
results.head()

,Id,SalePrice
0,1461,127234.636246
1,1462,160298.568758
2,1463,191059.604612
3,1464,188158.397500
4,1465,182978.153925


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Id'].plot(kind='hist', bins=20, title='Id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['SalePrice'].plot(kind='hist', bins=20, title='SalePrice')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='index', y='Id', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Id']
  ys = series['SalePrice']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('Id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Id')
_ = plt.ylabel('SalePrice')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Id']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Id'}, axis=1)
              .sort_values('Id', ascending=True))
  xs = counted['Id']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('Id', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Id')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_9['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['Id'].plot(kind='line', figsize=(8, 4), title='Id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['SalePrice'].plot(kind='line', figsize=(8, 4), title='SalePrice')
plt.gca().spines[['top', 'right']].set_visible(False)

In [40]:
results.to_csv('submissions.csv', index=False)